In [1]:
import random
import itertools

import pandas as pd
import numpy as np

from operator import itemgetter

# Genetic Search Algorithm

In [2]:
def point_mutation(lineup, rate):
    '''
    Description:
        Randomly change a bit inside of a particular child

    Input:
        lineup(list): list of players
        rate(dbl): the rate at which the mutation happens

    Output:
        child(list): the bitstring with possible mutations from the original
    '''
    # Randomly change biots within the child 
    child = lineup.copy()
    for idx, j in enumerate(lineup):
        if random.uniform(0, 1) < rate:
            if child[idx] == 0:
                child[idx] = 1
            else:
                child[idx] = 0

    return child

def crossover(parent1, parent2, rate):
    '''
    Description:
        mutate a new child based off of the combination of two parents.

    Input:
        parent1(list): list of bitstrings
        parent2(list): list of bitstrings
        rate(dbl): the rate at which the muatation happens. If a random number
        is greater than the rate, no mutation will occur.

    Output:
        a possible combination of the two parents.
    '''
    # If the mutation rate is lower than the overall rate,
    # take part of parent 1 and part of parent two and combine them 
    # into a new child
    if random.uniform(0, 1) >= rate:
        return parent1
    else:
        point = 1+random.randint(0, (len(parent1)-2))
        return parent1[0:point]+parent2[point:len(parent2)]

def reproduce(selected, pop_size, p_cross, p_mutation):
    '''
    Description:
        Take parent strings a combine them combine/mutate them into children strings.

    Input:
        selected(list): list of dictionaries that are potential best strings
        pop_size(int): number of candiadtes to consider
        p_cross(dbl): rate at which the parent to parent crossover happens
        p_mutation(dbl): rate at which the single point mutation will happen

    Output:
        children(list): list of all the children created from the parents.
    '''
    children = []
    # Selects the items before (Odd indicies) and after (even indicies) 
    # for mutation later
    for i, p1 in enumerate(selected):
        if i % 2 == 0:
            p2 = selected[i+1]
        else:
            p2 = selected[i-1]

        if i == (len(selected)-1):
            p2 = selected[0]

        child = {}
        # Tries to combine the two parents into new children
        child['lineup'] = crossover(p1['lineup'], p2['lineup'], p_cross)

        # Randomly mutates points within the child
        child['lineup'] = point_mutation(child['lineup'], p_mutation)
        children.append(child)
    
        if len(children) == pop_size:
            break

    return children

def binary_tournament(pop):
    '''
    Description:
        The function will randomly select two different lists from the population pool.
        It will then compared the cost of the two list against each other. Which ever
        list has a better cost, they get selected and move on in the selection process.

    Input:
        pop(list): List of dictionaries containing the cost and bits for each item under
        consideration

    Output:
        dictionary contating the list that won the "tournament"

        {bitstring: [],
         fitness: int}
    '''
    # Randomly select two bitstrings from the overall list
    i, j = random.randint(0, (len(pop)-1)), random.randint(0, (len(pop)-1))

    # Check to see if the indicies are the same
    while i == j:
        j =  random.randint(0, len(pop)-1)

    # Select the string with the highest cost to move forward.
    if pop[i]['fitness'] >= pop[j]['fitness']:
        return pop[i]
    else: 
        return pop[j]

def onemax(lineup, player_scores_weeks, week):
    '''
    The cost of the algorithm is the sum of the bits (Trying to obtain all 1's)
    '''
    scores = [player_scores_weeks[week][i] for i in lineup]
    return sum(scores)

def random_lineup(formation, person_lookup, week):
    '''
    Description:
        Create a random list of players

    Input:
        formation(tuple): tuple containing the number of defenders, midfielders and forwards in the formation
        person_lookup(dict): Dictionary containing the names of the players for each position

    Output:
        A list of randomly generated players.
    '''
    goalie = 1
    defense, midfield, forward = formation

    goalie_idx = random.sample(range(0, (len(person_lookup[week]['GKP'])-1)), 1)
    defense_idx = random.sample(range(0, (len(person_lookup[week]['DEF'])-1)), defense)
    midfield_idx = random.sample(range(0, (len(person_lookup[week]['MID'])-1)), midfield)
    forward_idx = random.sample(range(0, (len(person_lookup[week]['FWD'])-1)), forward)

    goalies = list(person_lookup[week]['GKP'][goalie_idx])
    defenders = list(person_lookup[week]['DEF'][defense_idx])
    midies = list(person_lookup[week]['MID'][midfield_idx])
    forwards = list(person_lookup[week]['FWD'][forward_idx])

    return goalies + defenders + midies + forwards

In [21]:
def search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, week):
    '''
Description:
    This function contians the main logic of the program. It will search through
    the various combinations of children and parents to find a soultion that will
    optimize the given cost functions. In this case, the optimal solution is a 
    list of all ones.

Input:
    max_gens(int): The number of generations to loop through to find an optimal
    solution.
    num_bits(int): The number of bits contained in the list
    pop_size(int): The number of children to be children to be considered against the parent.
    p_crossover(dbl): THe probability of keeping the parent vs mixing aprent and child.
        ex) p_crossover = .99; probability of keeping parent to next generation = .01
    p_mutation(dbl): The probability of changing a single position in a list.

Output:
    best(list): A list of a possible optimal solution.  
    '''
    # Create a random set of possible solutions
    population = [{'lineup':random_lineup(formation, person_lookup, week)} for i in range(pop_size)]
    
    # Find the "cost" of each colution
    for idx,_ in enumerate(population):
        population[idx]['fitness'] = onemax(population[idx]['lineup'], player_scores_weeks, week)

    # # Sort the lists to find the solution with the highest cost, and keep the largest as the best solution
    # best = sorted(population, key=itemgetter('fitness'), reverse=True)[0]

    # # Loop through the number of generations
    # for gen in range(max_gens):
    #     # Randomly select possible offspring
    #     selected = [binary_tournament(population) for _ in range(len(population))]
    #     # Reproduce children based off of the parent 
    #     children = reproduce(selected, pop_size, p_crossover, p_mutation)

    #     for idx,_ in enumerate(children):
    #         children[idx]['fitness'] = onemax(children[idx]['lineup'])

    #     sort_children =  sorted(children, key=itemgetter('fitness'), reverse=True) 

    #     # If the best child has a higher cost than current, replace the current with child
    #     if sort_children[0]['fitness'] >= best['fitness']:
    #         best = sort_children[0].copy()

    #     population = sort_children.copy()
    #     print('Gen: {0}, Fitness: {1}, Best: {2} '.format(gen, best['fitness'], best['lineup']))

    #     # Break out of the loop if the found the best solution before the end of the loop
    #     if sum(best['lineup']) == num_bits:
    #         break

    return population

In [22]:
formation = (4,3,3)
max_gens = 100
pop_size = 10
p_crossover = .99
p_mutation = 1.0/64

search(max_gens, formation, pop_size, p_crossover, p_mutation, person_lookup, player_scores_weeks, 0)

[{'lineup': ['Hennessey, CRY',
   'Mings, BOU',
   'Hadergjonaj, HUD',
   'Baines, EVE',
   'McQueen, SOU',
   'Ki Sung-yueng, NEW',
   'Choudhury, LEI',
   'Capoue, WAT',
   'Okaka, WAT',
   'Sinclair, WAT',
   'Sturridge, LIV'],
  'fitness': 3293.5629088157275},
 {'lineup': ['Forster, SOU',
   'Jagielka, EVE',
   'Robertson, LIV',
   'Bednarek, SOU',
   'Blind, MUN',
   'Noble, WHU',
   'Stanislas, BOU',
   'Son, TOT',
   'Barnes, BUR',
   'Arnautovic, WHU',
   'Ings, LIV'],
  'fitness': 8308.34158082253},
 {'lineup': ['Gomes, WAT',
   'Kaboul, WAT',
   'Delph, MCI',
   'Taylor, BUR',
   'Steve Cook, BOU',
   'Lucas Moura, TOT',
   'Ampadu, CHE',
   'Mata, MUN',
   'Locadia, BHA',
   'Hugill, WHU',
   'Giroud, CHE'],
  'fitness': 4539.034050774496},
 {'lineup': ['Speroni, CRY',
   'Malone, HUD',
   'Tomkins, CRY',
   'Smalling, MUN',
   'Lejeune, NEW',
   'Townsend, CRY',
   'Matic, MUN',
   'Antonio, WHU',
   'Llorente, TOT',
   'King, BOU',
   'Niasse, EVE'],
  'fitness': 4876.0139

# Data Prep

In [5]:
def normalize_data(col):
    '''
    Helper function to normalize a array of data betwene 0 (minimum) and 1 (maximum)

    Input:
        col(list/series): columns from a dataframe that is to be normalized

    Output:
        Normalized array with values between 0 and 1
    '''
    return (col-np.min(col))/(np.max(col)-np.min(col))

def scoring(df, label, week):
    '''
    This function will go through and calculate the scores for each player at their 
    given poisition. The scores are used by the algorithms cost function to figure out
    what the total "cost" would be to add a patricular player to the roster.

    Input:
        df(pd.dataframe): List of all the players for a given position
        label(str): label for the position of the players in ther
        week(int): the week of the season the daya is in.
            Week 0 is "preseaon" and uses projected numbers. The coefficients
            of the equations have to be addujsted accordingly
    
    Output:
        a dataframe with the calculated scores appended on the end.
    '''

    data = df.copy()

    # a Multiplier for the coefficients (There are 38 games in the average EPL season)
    if week == 0:
        ceof_multi = 38
    else:
        ceof_multi = 1

    # Equation to calculate the goalkeepers scores
    if label == 'GKP':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Scores'] = 10*ceof_multi - \
                          1.5*ceof_multi*data['Goals_conceded'] + \
                          2.0*ceof_multi*data['Penalties_saved'] + \
                          ceof_multi*data['Saves'] - \
                          3.0*ceof_multi*data['Red_cards'] + \
                          2.5*ceof_multi*data['Norm_Influence']

        # If the score is equal to the intercept, then the player has no stats to be considered
        # Therefore they will not be on a roster.
        
        return data[data['Scores'] != 10*ceof_multi]

    # Equation to calculate the defenders scores
    elif label == 'DEF':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Scores'] = 10*ceof_multi - \
                            ceof_multi*data['Goals_conceded'] + \
                          5*ceof_multi*data['Goals_scored'] + \
                          2.5*ceof_multi*data['Norm_Influence'] - \
                          1.5*ceof_multi*data['Norm_Threat'] + \
                          1.5*ceof_multi*data['Norm_Creativity'] - \
                          2.0*ceof_multi*data['Red_cards'] + \
                          ceof_multi*data['Assists'] - \
                          2.0*ceof_multi*data['Own_goals']  
        return data[data['Scores'] != 10*ceof_multi]     

    # Equation to calculate the Midfielders scores
    elif label == 'MID':
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Scores'] = 10*ceof_multi - \
                        ceof_multi*data['Goals_conceded'] + \
                        2.0*ceof_multi*data['Goals_scored'] + \
                        2.5*ceof_multi*data['Norm_Influence'] + \
                        1.5*ceof_multi*data['Norm_Threat'] + \
                        5.0*ceof_multi*data['Norm_Creativity'] - \
                        2.0*ceof_multi*data['Red_cards'] + \
                        2.0*ceof_multi*data['Assists'] - \
                        2.0*ceof_multi*data['Own_goals']

        return data[data['Scores'] != 10*ceof_multi] 

    # Equation to calculate the Forwards scores
    else:
        data['Norm_Influence'] = 5.0*normalize_data(data['Influence'])
        data['Norm_Creativity'] = 5.0*normalize_data(data['Creativity'])
        data['Norm_Threat'] = 5.0*normalize_data(data['Threat'])
        data['Scores'] = 10*ceof_multi + \
                            2.0*ceof_multi*data['Goals_scored'] + \
                            2.5*ceof_multi*data['Norm_Influence'] + \
                            1.5*ceof_multi*data['Norm_Creativity'] + \
                            5.0*ceof_multi*data['Norm_Threat'] - \
                            2.5*ceof_multi*data['Red_cards'] + \
                            ceof_multi*data['Assists']
        return data[data['Scores'] != 10*ceof_multi] 

In [6]:
PATH = 'https://raw.githubusercontent.com/DanielBrooks253/Kaggle/main/Soccer_Team/Data/'
filenames = ['FPL_2018_19_Wk0.csv', # Projected Scores
             'FPL_2018_19_Wk1.csv',
             'FPL_2018_19_Wk2.csv',
             'FPL_2018_19_Wk3.csv',
             'FPL_2018_19_Wk4.csv',
             'FPL_2018_19_Wk5.csv',
             'FPL_2018_19_Wk6.csv',
             'FPL_2018_19_Wk7.csv',
             'FPL_2018_19_Wk8.csv',
             'FPL_2018_19_Wk9.csv',
             'FPL_2018_19_Wk10.csv',
             'FPL_2018_19_Wk11.csv']

person_lookup = {'DEF': [],
                 'GKP': [],
                 'MID': [],
                 'FWD': []}

# Get the individual weeks and also the overall combined dataset
weeks = [pd.read_csv(PATH + i) for i in filenames]
adjust_names = weeks.copy()

# Some of the names of the players repeat.
# This causes issues when trying to create dictionary keys later on.
# THerefore I concatenated the team name with the last name to make a unique key.
for idx, _ in enumerate(adjust_names):
    adjust_names[idx]['New_Name'] = adjust_names[idx]['Name'] + ', ' + adjust_names[idx]['Team']  

In [7]:
# Get the scores for all of the positions for all of the available weeks
list_player_score_df = [pd.concat(list(map(lambda x: scoring(adjust_names[i][adjust_names[i]['Position'] == x], x, i), ['DEF', 'FWD', 'GKP', 'MID'])), axis=0)
                           for i in range(0, 12)]

# Create a list of dictionaries for each week.
# The dictionary maps the players name to their scores
player_scores_weeks = [dict(zip(i['New_Name'], i['Scores'])) for i in list_player_score_df]

# Filter the weeks to only players who played 
player_score_keys = [list(i.keys()) for i in player_scores_weeks]
filtered_weeks = [i[i['New_Name'].isin(j)] for i,j in zip(adjust_names, player_score_keys)]

In [8]:
# Get the mapping from the position to the person
filter_person = [list(map(lambda x: filtered_weeks[i][filtered_weeks[i]['Position'] == x], ['DEF', 'FWD', 'GKP', 'MID'])) for i in range(0, 12)]
person_lookup = [{j['Position'].unique()[0]:np.array(j['New_Name']) for j in i} for i in filter_person]

# Different Formations (# Defenders - # Midfielders - # Forwards)

## 4-3-3

## 3-6-1

## 4-5-1

## 4-4-2

# Best Team for Lowest Player Cost